# Catalog Netflix

Autor: Thiago Vilarinho Lemes\
Projeto: ETL no Databricks utilizando Catalog \
Data: 14/09/2025

In [46]:
from databricks.connect import DatabricksSession
from databricks.sdk.runtime import dbutils
from dotenv import load_dotenv
import os

In [ ]:
# Carrega as variáveis de ambiente do arquivo .env

load_dotenv()

catalog_name = os.getenv("NAME_CATALOG")

schema_bronze = os.getenv("NAME_SCHEMA_BRONZE")
table_bronze = os.getenv("NAME_TABLE_BRONZE")

schema_silver = os.getenv("NAME_SCHEMA_SILVER")
table_silver = os.getenv("NAME_TABLE_SILVER")

schema_gold = os.getenv("NAME_SCHEMA_GOLD")
table_gold = os.getenv("NAME_TABLE_GOLD")

schema_raw = os.getenv("NAME_SCHEMA_RAW")
bucket_raw = os.getenv("NAME_STORAGE")

In [48]:
# Cria a sessão Spark usando Databricks Connect
spark = DatabricksSession.builder.getOrCreate()

In [49]:
# Substitua pelo nome do catálogo que você quer criar
catalog_name = catalog_name

In [50]:
# Comando SQL para criar o catálogo
spark.sql(f"""
CREATE CATALOG IF NOT EXISTS {catalog_name}
COMMENT 'Catálogo criado via Databricks Connect'
""")
print(f"Catálogo {catalog_name} criado com sucesso!")

Catálogo catalogo_netflix_movies criado com sucesso!


In [51]:
# Criando um Schema dentro do Catálogo

# Nome do schema 
schema_name = "bronze"

spark.sql(f"""
CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}
COMMENT 'Schema Bronze para dados raw'
""")

print(f"Schema '{schema_bronze}' criado no catálogo {catalog_name}.")


Schema 'bronze' criado no catálogo catalogo_netflix_movies.


In [52]:
# Criando um Schema dentro do Catálogo

# Nome do schema 
schema_name = "silver"

spark.sql(f"""
CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}
COMMENT 'Schema Bronze para dados raw'
""")

print(f"Schema '{schema_silver}' criado no catálogo {catalog_name}.")

Schema 'silver' criado no catálogo catalogo_netflix_movies.


In [53]:
# Criando um Schema dentro do Catálogo

# Nome do schema 
schema_name = "gold"

spark.sql(f"""
CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}
COMMENT 'Schema Bronze para dados raw'
""")

print(f"Schema '{schema_gold}' criado no catálogo {catalog_name}.")

Schema 'gold' criado no catálogo catalogo_netflix_movies.


Os códigos comentados abaixo é para criar a camada Raw, mas ele não foi necessária

In [ ]:
# Criando um Schema dentro do Catálogo

# Nome do schema 
# schema_name = "raw"

# spark.sql(f"""
# CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}
# COMMENT 'Schema Bronze para dados raw'
# """)

# print(f"Schema '{schema_raw}' criado no catálogo {catalog_name}.")

In [ ]:
# Criando um Volume dentro do Schema Raw para os datasets brutos
# spark.sql(f"""
#     CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_raw}.{bucket_raw}
#     COMMENT 'Volume para armazenar arquivos brutos do Thiago'
#     """)

In [55]:
# Verifica se o catálogo foi criado
catalogs = spark.sql("SHOW CATALOGS").collect()
if (catalog_name,) in catalogs:
    print(f"Catálogo '{catalog_name}' criado com sucesso!")
else:
    print(f"Catálogo '{catalog_name}' não foi encontrado.")

Catálogo 'catalogo_netflix_movies' criado com sucesso!


In [56]:
# Verifica os schemas dentro do catálogo 
schemas = spark.sql(f"SHOW SCHEMAS IN {catalog_name}").collect()
print(f"Schemas no catálogo {catalog_name}:")
print("-" * 45)
for schema in schemas:
    print(f" - {schema.databaseName}")

Schemas no catálogo catalogo_netflix_movies:
---------------------------------------------
 - bronze
 - default
 - gold
 - information_schema
 - raw
 - silver


OBS.: As células para baixa serve somente como referência caso precise

In [ ]:
import requests
import os

DATABRICKS_HOST = "https://dbc-ed3459db-1454.cloud.databricks.com"
DATABRICKS_TOKEN = os.getenv("DATABRICKS_TOKEN")

local_file = "../../dataset/netflix_v01.parquet"
volume_path = f"/Volumes/{catalog_name}/{schema_raw}/{bucket_raw}/netflix_v01.parquet"

headers = {"Authorization": f"Bearer {DATABRICKS_TOKEN}"}
url = f"{DATABRICKS_HOST}/api/2.0/files/upload"

with open(local_file, "rb") as f:
    response = requests.post(
        url,
        headers=headers,
        params={"path": volume_path, "overwrite": "true"},
        files={"contents": f}
    )

print(response.status_code, response.text)

In [45]:


# 2. Ler o parquet local
df = spark.read.parquet("../../dataset/netflix_v01.parquet")

# 3. Escrever no volume
output_path = f"dbfs:/Volumes/{catalog_name}/{schema_raw}/{bucket_raw}/netflix_v01.parquet"

df.write.mode("overwrite").parquet(output_path)

print(f"✅ Arquivo salvo em {output_path}")


IllegalArgumentException: Path must be absolute: ../../dataset/netflix_v01.parquet

JVM stacktrace:
java.lang.IllegalArgumentException
	at com.databricks.common.path.AbstractPath$.fromHadoopPath(AbstractPath.scala:114)
	at com.databricks.backend.daemon.data.filesystem.MountEntryResolver.resolve(MountEntryResolver.scala:52)
	at com.databricks.backend.daemon.data.client.DBFSOnUCFileSystemResolverImpl.resolveWithSam(DBFSOnUCFileSystemResolverImpl.scala:161)
	at com.databricks.backend.daemon.data.client.DBFSOnUCFileSystemResolverImpl.resolveAndGetFileSystem(DBFSOnUCFileSystemResolverImpl.scala:217)
	at com.databricks.backend.daemon.data.client.DBFSV2.resolveAndGetFileSystem(DatabricksFileSystemV2.scala:146)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.resolve(DatabricksFileSystemV2.scala:774)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$2(DatabricksFileSystemV2.scala:1187)
	at com.databricks.s3a.S3AExceptionUtils$.conve